In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('C:\\Users\\yoons\DeepLearning\Kaggle Study\Image Classification\input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\input\test_img\0.jpg
C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\input\test_img\1.jpg
C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\input\test_img\10.jpg
C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\input\test_img\100.jpg
C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\input\test_img\1000.jpg
C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\input\test_img\1001.jpg
C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\input\test_img\1002.jpg
C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\input\test_img\1003.jpg
C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\input\test_img\1004.jpg
C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\input\test_img\1005.jpg
C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\input\test_img\1006.jpg
C:\Users\yoons\DeepLearning\Kaggle Study\Image Classification\i

In [2]:
#!pip install timm

In [3]:
import timm

avail_pretrained_models = timm.list_models(pretrained=True)
for model in avail_pretrained_models:
    print(model)

adv_inception_v3
bat_resnext26ts
beit_base_patch16_224
beit_base_patch16_224_in22k
beit_base_patch16_384
beit_large_patch16_224
beit_large_patch16_224_in22k
beit_large_patch16_384
beit_large_patch16_512
botnet26t_256
cait_m36_384
cait_m48_448
cait_s24_224
cait_s24_384
cait_s36_384
cait_xs24_384
cait_xxs24_224
cait_xxs24_384
cait_xxs36_224
cait_xxs36_384
coat_lite_mini
coat_lite_small
coat_lite_tiny
coat_mini
coat_tiny
convit_base
convit_small
convit_tiny
convmixer_768_32
convmixer_1024_20_ks9_p14
convmixer_1536_20
convnext_base
convnext_base_384_in22ft1k
convnext_base_in22ft1k
convnext_base_in22k
convnext_large
convnext_large_384_in22ft1k
convnext_large_in22ft1k
convnext_large_in22k
convnext_small
convnext_tiny
convnext_xlarge_384_in22ft1k
convnext_xlarge_in22ft1k
convnext_xlarge_in22k
crossvit_9_240
crossvit_9_dagger_240
crossvit_15_240
crossvit_15_dagger_240
crossvit_15_dagger_408
crossvit_18_240
crossvit_18_dagger_240
crossvit_18_dagger_408
crossvit_base_240
crossvit_small_240
cross

In [4]:
import torch
import random
import tqdm
import cv2
import os
import timm
import copy
import torchvision
import PIL
import matplotlib.pyplot as plt
from timm.data.auto_augment import auto_augment_transform

submit = pd.read_csv('Sample_Submit.csv')

In [5]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

In [6]:
seed_everything()

class Dataset(torch.utils.data.Dataset):
    def __init__(self,split,path,transform,data_aug=None):
        super(Dataset,self).__init__()
        self.split = split
        self.root_path = path
        self.transform = transform
        self.augmentation = data_aug
        self.video_list = list()
        self.imgs = list()
        if split == 'train':
            self.labels = list()
            for cls in os.listdir(path):
                #print(path)
                #print(os.path.join(path,cls))
                #print(os.listdir(os.path.join(path,cls)))
                self.imgs = os.listdir(os.path.join(path,cls))
                self.imgs[0] = cls + '\\' + self.imgs[0]
                self.imgs[1] = cls + '\\' + self.imgs[1]
                self.imgs[2] = cls + '\\' + self.imgs[2]
                #print(self.imgs)
                self.video_list.extend(os.listdir(os.path.join(path,cls)))
                self.labels.extend(np.full((len(os.listdir(os.path.join(path,cls)))),int(cls)))
                #print(self.video_list)
        elif split == 'test':
            self.video_list = os.listdir(path)
        #print(self.video_list)    
    def __len__(self):
        return len(self.video_list)
    
    def __getitem__(self,index):
        if self.split == 'train':
            img = PIL.Image.open(os.path.join(self.root_path,str(self.labels[index]),self.video_list[index]))
            img = img.convert('RGB')
            if self.augmentation is not None:
                img = self.augmentation(img)
            img = self.transform(img)
            return img,self.labels[index]
        
        elif self.split == 'test':
            img = PIL.Image.open(os.path.join(self.root_path,str(index)+'.jpg'))
            img = img.convert('RGB')
            img = self.transform(img)
            return img

In [7]:
import torchvision.transforms as transforms

seed_everything()

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(260),
        transforms.RandomRotation(degrees=(-10,10)),
        transforms.RandomAffine(degrees=(-16, 16), translate=(.1,.1)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


train_dataset = Dataset(path='C:\\Users\\yoons\\DeepLearning\\Kaggle Study\\Image Classification\\input\\train_img',
                        split = 'train',
                        transform = transform,
                        data_aug = None)

#print("Train Dataset loaded")
#print(train_dataset)
test_dataset = Dataset(path='C:\\Users\\yoons\\DeepLearning\\Kaggle Study\\Image Classification\\input\\test_img',
                       split = 'test',
                       transform = transform)
#print("Test Dataset loaded")
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True,
                                           num_workers=4,
                                           drop_last = True
                                          )


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=64,
                                          shuffle=False,
                                          num_workers=0,
                                         )

In [ ]:
seed_everything()

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # 갱신이 될 때까지 잠시 기다립니다.


# 학습 데이터의 배치를 얻습니다.
inputs, classes = next(iter(train_loader))
inputs2 = next(iter(train_loader))
# 배치로부터 격자 형태의 이미지를 만듭니다.
out = torchvision.utils.make_grid(inputs)
out2 = torchvision.utils.make_grid(inputs2)
imshow(out)
imshow(out2)

In [ ]:
seed_everything()

model = timm.create_model(model_name = 'efficientnet_b2', 
                          pretrained = True,
                          num_classes=900)
model = model.cuda()
model.train()

Loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.1,momentum=0.9)

Total_Epoch = 25

for epoch in range(Total_Epoch):
    epoch_loss = 0
    epoch_accuracy = 0
    for X,Y in tqdm.tqdm(train_loader):
        X = torch.FloatTensor(X).cuda()
        Y = torch.LongTensor(Y).cuda()

        output = model(X)
        Cost = Loss(output,Y)

        optimizer.zero_grad()
        Cost.backward()
        optimizer.step()

        hypothesis = torch.nn.functional.softmax(output,dim=1)
        predict = torch.argmax(hypothesis,dim=1)
        correct_predict = predict == Y

        epoch_accuracy = epoch_accuracy + correct_predict.sum()
        epoch_loss = epoch_loss + Cost.item()

    if epoch % 1 == 0:
        print(" Epoch : {:4d} , Cost : {:.2f} , Accuracy : {:.4f}".format(epoch,epoch_loss,epoch_accuracy/len(train_loader.dataset)))
        

In [ ]:
seed_everything()

total_prediction = list()
model.eval()
with torch.no_grad():
    for X in tqdm.tqdm(test_loader):
        X = torch.FloatTensor(X).cuda()
        output = model(X)
        hypothesis = torch.nn.functional.softmax(output,dim=1)
        predict = torch.argmax(hypothesis,dim=1)
        total_prediction.extend(predict.cpu().numpy())
    submit['label'] = total_prediction
    submit.to_csv('submit.csv',index=False)